In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [44]:
df = pd.read_csv('processed.csv')
df.head()

,Unnamed: 0,difficulty,size,gas_limit,transaction_count,gas,past_min,past_mean,past_max,block_min
0,0,0.497304,1.525696,1.376636,0.768851,-0.306444,-0.471469,-0.552406,-0.503274,0.696954
1,1,-0.175685,-0.151892,0.634122,-0.146996,0.373021,0.021040,0.091190,0.868203,-0.093861
2,2,0.074976,-1.414733,-0.108392,-0.544231,-0.382288,-0.202041,-0.555886,-0.473770,-0.357466
3,3,0.730501,0.085234,0.634122,-0.334579,-0.348186,0.036881,-0.276384,-0.418639,-0.291564
4,4,1.549208,0.792288,0.634122,0.934365,-0.309031,-0.300726,-0.273407,-0.412249,0.433349


In [45]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [46]:
df.head()

,difficulty,size,gas_limit,transaction_count,gas,past_min,past_mean,past_max,block_min
0,0.497304,1.525696,1.376636,0.768851,-0.306444,-0.471469,-0.552406,-0.503274,0.696954
1,-0.175685,-0.151892,0.634122,-0.146996,0.373021,0.021040,0.091190,0.868203,-0.093861
2,0.074976,-1.414733,-0.108392,-0.544231,-0.382288,-0.202041,-0.555886,-0.473770,-0.357466
3,0.730501,0.085234,0.634122,-0.334579,-0.348186,0.036881,-0.276384,-0.418639,-0.291564
4,1.549208,0.792288,0.634122,0.934365,-0.309031,-0.300726,-0.273407,-0.412249,0.433349


## Train-Test split

In [47]:
from sklearn.model_selection import train_test_split
x = df.drop(['block_min'], axis=1)
y = df['block_min']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

## Gridsearch to tune parameters

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

clf = SVR()
kernel = ['linear', 'sigmoid', 'rbf']
C = [1, 5, 10]
parameters = {'kernel': kernel ,'C': C}
model = GridSearchCV(estimator=clf, param_grid=parameters, cv=3, scoring='r2', return_train_score=True, verbose=1)
model.fit(x_train[:10000], y_train[:10000])

Fitting 3 folds for each of 9 candidates, totalling 27 fits


In [ ]:
print("Model with best parameters :\n", model.best_params_)
train_auc = model.cv_results_['mean_train_score'].reshape(len(kernel), len(C))
cv_auc = model.cv_results_['mean_test_score'].reshape(len(kernel), len(C))

f, axes = plt.subplots(1, 2,figsize=(15,5))

for i in range(2):
    title = train_auc if i == 0 else cv_auc
    sns.heatmap(title, xticklabels=no_of_estimators, yticklabels=maximumdepth, annot=True, ax=axes[i])
    axes[i].set_title(f"Gridearch {'Train' if i==0 else 'CV'}")
    axes[i].set_xlabel("no_of_estimators")
    axes[i].set_ylabel("maximum_depth")

## Training the model

In [ ]:
svr = SVR(**model.best_params_)
svr.fit(x_train[:10000], y_train)

In [ ]:
y_pred = svr.predict(x_test)

In [ ]:
y_p_tr = svr.predict(x_train)

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,accuracy_score, r2_score
from math import sqrt

def calculateMetric(y_pred,y_test):
    print("MAE=",mean_absolute_error(y_test,y_pred))
    print("RMSE=",sqrt(mean_squared_error(y_test,y_pred)))
    print("R2 Score", r2_score(y_test,y_pred))

In [ ]:
calculateMetric(y_pred, y_test)

In [ ]:
calculateMetric(y_p_tr, y_train)